In [1]:
import chess
import chess.svg

from IPython.display import SVG

board = chess.Board()
SVG(chess.svg.board(board=board,size=400)) #show board at jupyter notebook

import pandas as pd
chess_game = pd.read_csv("dataset/games.csv", index_col=0)

dataset = []
#for i in range(1000):
for i in range(len(chess_game)):
    x = chess_game.iloc[i][11]
    x = x.split()
    array_move = []
    for j in range(349):
        list_square = []
        
        if len(x) < j+1:
            
            array_move.append([-1,-1])
        else:
            a = board.push_san(x[j])
            list_square.append(a.from_square)
            list_square.append(a.to_square)
            array_move.append(list_square)
    dataset.append(array_move)
    board = chess.Board()
    #print("%d 번째"%i, "게임 코드", array_move)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

x1_train = torch.FloatTensor(dataset[0])
x2_train = torch.FloatTensor(dataset[1])
x3_train = torch.FloatTensor(dataset[2])
y_train = torch.FloatTensor(dataset[3])
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w1,w2,w3,b], lr=1e-5)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.001 w2: 0.001 w3: 0.005 b: 0.000 Cost: 277.912598
Epoch  100/1000 w1: 0.055 w2: 0.077 w3: 0.369 b: 0.009 Cost: 132.083099
Epoch  200/1000 w1: 0.081 w2: 0.113 w3: 0.583 b: 0.014 Cost: 82.700829
Epoch  300/1000 w1: 0.091 w2: 0.128 w3: 0.709 b: 0.018 Cost: 65.793823
Epoch  400/1000 w1: 0.093 w2: 0.132 w3: 0.785 b: 0.021 Cost: 59.870506
Epoch  500/1000 w1: 0.091 w2: 0.129 w3: 0.831 b: 0.023 Cost: 57.697861
Epoch  600/1000 w1: 0.088 w2: 0.124 w3: 0.859 b: 0.025 Cost: 56.832031
Epoch  700/1000 w1: 0.083 w2: 0.117 w3: 0.877 b: 0.027 Cost: 56.440292
Epoch  800/1000 w1: 0.079 w2: 0.110 w3: 0.888 b: 0.028 Cost: 56.233856
Epoch  900/1000 w1: 0.074 w2: 0.103 w3: 0.896 b: 0.029 Cost: 56.108845
Epoch 1000/1000 w1: 0.071 w2: 0.097 w3: 0.901 b: 0.031 Cost: 56.025345


In [61]:
import numpy as np

x_raw = np.array(dataset[0:1000])
#x_raw = x_raw.transpose([1,2,0])
y_raw = [np.array(dataset[11554])]


x_train = torch.FloatTensor(x_raw)
y_train = torch.FloatTensor(y_raw)# which has full m349 moves, 13859 as well


print(x_train.shape)
print(y_train.shape)


torch.Size([1000, 349, 2])
torch.Size([1, 349, 2])


In [68]:
W = torch.zeros((1000, 1,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [69]:
optimizer = optim.SGD([W, b], lr=1e-5)

In [70]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b
    
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

RuntimeError: Expected tensor to have size 2 at dimension 1, but got size 1 for argument #2 'batch2' (while checking arguments for bmm)